# Federated Learning with Differential Privacy + Compression - HAR

This notebook implements **FedAvg with Differential Privacy and Communication Compression**.

**Experiment Configuration:**
- Dataset: UCI HAR (Human Activity Recognition)
- Training Mode: Federated Learning (FedAvg) + Differential Privacy
- **Privacy Budget (ε)**: [20, 50, 80] (lower = more privacy)
- **Non-IID (α)**: [0.1, 0.3, 1.0]
- **Compression**: With/Without (Top-K 10%)
- **Seeds**: [42, 123, 456]
- **Total Runs**: 54 (3 α × 3 ε × 3 seeds × 2 compression modes)
- Rounds: 100
- Clients: 10 (5 per round)

**Dataset Source:** https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones

**Compatible with:** Local and Kaggle

## 1. Environment Setup

In [ ]:
import os

IS_KAGGLE = os.path.exists('/kaggle/input')
DATA_PATH = '/kaggle/input/human-activity-recognition-with-smartphones' if IS_KAGGLE else '.'
print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"Data path: {DATA_PATH}")

## 2. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random
import json
from datetime import datetime
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PyTorch: {torch.__version__}, NumPy: {np.__version__}")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 3. Data Loading with Non-IID Partitioning

In [ ]:
class UCIHARDataLoader:
   def __init__(self, data_path='.'):
       self.data_path = data_path
       self.label_encoder = LabelEncoder()
       self.activity_labels = None

   def load_data(self):
       train_df = pd.read_csv(os.path.join(self.data_path, 'train.csv'))
       test_df = pd.read_csv(os.path.join(self.data_path, 'test.csv'))

       X_train = train_df.drop(['Activity', 'subject'], axis=1).values
       y_train = train_df['Activity'].values
       X_test = test_df.drop(['Activity', 'subject'], axis=1).values
       y_test = test_df['Activity'].values

       self.label_encoder.fit(np.concatenate([y_train, y_test]))
       y_train = self.label_encoder.transform(y_train)
       y_test = self.label_encoder.transform(y_test)

       self.activity_labels = {i: label for i, label in enumerate(self.label_encoder.classes_)}
       return X_train, y_train, X_test, y_test

   def partition_data_dirichlet(self, X, y, num_clients, alpha, seed=42):
       np.random.seed(seed)
       num_classes = len(np.unique(y))
       client_indices = {i: [] for i in range(num_clients)}

       class_indices = {k: [] for k in range(num_classes)}
       for idx, label in enumerate(y):
           class_indices[label].append(idx)

       for k in range(num_classes):
           np.random.shuffle(class_indices[k])
           proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
           proportions = proportions / proportions.sum()
           proportions = (np.cumsum(proportions) * len(class_indices[k])).astype(int)[:-1]
           splits = np.split(class_indices[k], proportions)
           for client_id, split in enumerate(splits):
               client_indices[client_id].extend(split)

       for client_id in range(num_clients):
           np.random.shuffle(client_indices[client_id])

       return client_indices

   def get_federated_dataloaders(self, num_clients, alpha, batch_size=64, seed=42):
       X_train, y_train, X_test, y_test = self.load_data()
       client_indices = self.partition_data_dirichlet(X_train, y_train, num_clients, alpha, seed)

       client_loaders = []
       for client_id in range(num_clients):
           indices = client_indices[client_id]
           X_client = torch.FloatTensor(X_train[indices])
           y_client = torch.LongTensor(y_train[indices])
           client_dataset = TensorDataset(X_client, y_client)
           client_loader = DataLoader(client_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
           client_loaders.append(client_loader)

       test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))
       test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

       return client_loaders, test_loader, X_train.shape[1], len(self.activity_labels)

data_loader = UCIHARDataLoader(DATA_PATH)
X_train, y_train, X_test, y_test = data_loader.load_data()
print(f"Train: {X_train.shape[0]}, Test: {X_test.shape[0]}, Features: {X_train.shape[1]}")

## 4. Model Architecture

In [ ]:
class HARModel(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dims=[256, 128, 64], dropout=0.5):
        super(HARModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, num_classes))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

## 5. Differential Privacy Mechanism

In [ ]:
class DifferentialPrivacy:
    """
    Implements Gaussian Mechanism for Differential Privacy
    """
    def __init__(self, epsilon, delta=1e-5, sensitivity=1.0):
        """
        Args:
            epsilon: Privacy budget (lower = more privacy)
            delta: Probability of privacy breach
            sensitivity: L2 sensitivity of the query (gradient clipping norm)
        """
        self.epsilon = epsilon
        self.delta = delta
        self.sensitivity = sensitivity
        self.sigma = self._calculate_noise_scale()
    
    def _calculate_noise_scale(self):
        """
        Calculate Gaussian noise scale using the Gaussian mechanism
        σ = (sensitivity * sqrt(2 * ln(1.25/δ))) / ε
        """
        return (self.sensitivity * np.sqrt(2 * np.log(1.25 / self.delta))) / self.epsilon
    
    def add_noise(self, model_state_dict):
        """
        Add Gaussian noise to model parameters
        """
        noisy_state = {}
        for key, param in model_state_dict.items():
            if 'weight' in key or 'bias' in key:
                # Add Gaussian noise
                noise = torch.randn_like(param) * self.sigma
                noisy_state[key] = param + noise
            else:
                # Don't add noise to batch norm statistics
                noisy_state[key] = param
        return noisy_state
    
    def clip_gradients(self, model, max_norm):
        """
        Clip gradients to bound sensitivity
        """
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

# Test DP mechanism
dp = DifferentialPrivacy(epsilon=50, delta=1e-5, sensitivity=1.0)
print(f"\nDP Configuration:")
print(f"  Epsilon (ε): {dp.epsilon}")
print(f"  Delta (δ): {dp.delta}")
print(f"  Noise scale (σ): {dp.sigma:.4f}")
print(f"  Lower ε = More Privacy, Higher Noise")

## 6. Communication Compression

In [ ]:
class ModelCompressor:
    def __init__(self, compression_ratio=0.1):
        self.compression_ratio = compression_ratio
    
    def compress_model(self, model_state_dict):
        compressed_state = {}
        for key, param in model_state_dict.items():
            if 'weight' in key or 'bias' in key:
                flat_param = param.flatten()
                k = max(1, int(flat_param.numel() * self.compression_ratio))
                values, indices = torch.topk(torch.abs(flat_param), k)
                compressed_param = torch.zeros_like(flat_param)
                compressed_param[indices] = flat_param[indices]
                compressed_state[key] = compressed_param.reshape(param.shape)
            else:
                compressed_state[key] = param
        return compressed_state
    
    def calculate_compression_stats(self, original_state, compressed_state):
        original_bytes = 0
        compressed_bytes = 0
        
        for key in original_state.keys():
            if 'weight' in key or 'bias' in key:
                original_bytes += original_state[key].numel() * 4
                non_zero = torch.count_nonzero(compressed_state[key])
                compressed_bytes += non_zero.item() * 8  # value + index
        
        compression_ratio = compressed_bytes / original_bytes if original_bytes > 0 else 0
        return {
            'original_bytes': original_bytes,
            'compressed_bytes': compressed_bytes,
            'compression_ratio': compression_ratio,
            'savings_percent': (1 - compression_ratio) * 100
        }

## 7. FedAvg + DP + Compression

In [ ]:
def train_client(model, train_loader, criterion, optimizer, device, clip_norm=None, epochs=1):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping for DP
            if clip_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
            
            optimizer.step()
    return model.state_dict()

def aggregate_models(global_model, client_models, client_weights):
    global_dict = global_model.state_dict()
    for key in global_dict.keys():
        global_dict[key] = torch.stack(
            [client_models[i][key].float() * client_weights[i] for i in range(len(client_models))], 
            dim=0
        ).sum(dim=0)
    global_model.load_state_dict(global_dict)
    return global_model

def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = running_loss / len(all_labels)
    test_acc = accuracy_score(all_labels, all_preds)
    test_f1 = f1_score(all_labels, all_preds, average='weighted')
    return test_loss, test_acc, test_f1, all_preds, all_labels

def federated_training_dp(client_loaders, test_loader, input_dim, num_classes, 
                         num_rounds, clients_per_round, learning_rate, weight_decay,
                         epsilon, delta, clip_norm, 
                         use_compression, compression_ratio,
                         device, seed=42, verbose=True):
    set_seed(seed)
    
    global_model = HARModel(input_dim, num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    dp_mechanism = DifferentialPrivacy(epsilon=epsilon, delta=delta, sensitivity=clip_norm)
    compressor = ModelCompressor(compression_ratio) if use_compression else None
    
    history = {
        'test_loss': [],
        'test_acc': [],
        'test_f1': [],
        'communication_bytes': []
    }
    
    best_acc = 0.0
    total_communication_bytes = 0
    
    for round_num in range(num_rounds):
        np.random.seed(seed + round_num)
        selected_clients = np.random.choice(len(client_loaders), clients_per_round, replace=False)
        
        client_models = []
        client_weights = []
        round_bytes = 0
        
        for client_id in selected_clients:
            client_model = HARModel(input_dim, num_classes).to(device)
            client_model.load_state_dict(global_model.state_dict())
            
            optimizer = optim.Adam(client_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
            
            # Train with gradient clipping
            client_state = train_client(client_model, client_loaders[client_id], 
                                       criterion, optimizer, device, 
                                       clip_norm=clip_norm, epochs=1)
            
            # Add DP noise
            client_state = dp_mechanism.add_noise(client_state)
            
            # Apply compression if enabled
            if use_compression:
                compressed_state = compressor.compress_model(client_state)
                stats = compressor.calculate_compression_stats(client_state, compressed_state)
                round_bytes += stats['compressed_bytes']
                client_state = compressed_state
            else:
                for param in client_state.values():
                    round_bytes += param.numel() * 4
            
            client_models.append(client_state)
            client_weights.append(len(client_loaders[client_id].dataset))
        
        total_communication_bytes += round_bytes
        
        # Aggregate
        total_size = sum(client_weights)
        client_weights = [w / total_size for w in client_weights]
        global_model = aggregate_models(global_model, client_models, client_weights)
        
        # Evaluate
        test_loss, test_acc, test_f1, _, _ = evaluate_model(global_model, test_loader, criterion, device)
        
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['test_f1'].append(test_f1)
        history['communication_bytes'].append(total_communication_bytes)
        
        if test_acc > best_acc:
            best_acc = test_acc
        
        if verbose and (round_num + 1) % 10 == 0:
            comm_mb = total_communication_bytes / (1024 * 1024)
            print(f'Round [{round_num+1:3d}/{num_rounds}] '
                  f'Acc: {test_acc:.4f}, F1: {test_f1:.4f}, Comm: {comm_mb:.2f} MB')
    
    test_loss, test_acc, test_f1, predictions, true_labels = evaluate_model(global_model, test_loader, criterion, device)
    
    return history, best_acc, test_loss, test_acc, test_f1, predictions, true_labels, total_communication_bytes

## 8. Experiment Configuration

In [ ]:
EXPERIMENT_CONFIG = {
    'experiment_type': 'fedavg_dp_with_compression',
    'dataset': 'UCI_HAR',
    'alpha_values': [0.1, 0.3, 1.0],
    'epsilon_values': [20, 50, 80],
    'seeds': [42, 123, 456],
    'compression_modes': [False, True],
    'delta': 1e-5,
    'clip_norm': 1.0,
    'compression_ratio': 0.1,
    'num_clients': 10,
    'clients_per_round': 5,
    'num_rounds': 100,
    'batch_size': 64,
    'learning_rate': 0.001,
    'weight_decay': 1e-4
}

print("Experiment Configuration:")
print("="*60)
print(json.dumps(EXPERIMENT_CONFIG, indent=2))
print("="*60)
total_exp = (len(EXPERIMENT_CONFIG['alpha_values']) * 
             len(EXPERIMENT_CONFIG['epsilon_values']) * 
             len(EXPERIMENT_CONFIG['seeds']) * 
             len(EXPERIMENT_CONFIG['compression_modes']))
print(f"\nTotal Experiments: {total_exp}")
print(f"  Without Compression: 27 (3 α × 3 ε × 3 seeds)")
print(f"  With Compression: 27 (3 α × 3 ε × 3 seeds)")

## 9. Run All Experiments

In [ ]:
all_results = []
experiment_num = 0

for use_compression in EXPERIMENT_CONFIG['compression_modes']:
    for alpha in EXPERIMENT_CONFIG['alpha_values']:
        for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
            for seed in EXPERIMENT_CONFIG['seeds']:
                experiment_num += 1
                comp_str = "WITH" if use_compression else "WITHOUT"
                print(f"\n{'='*80}")
                print(f" Exp {experiment_num}/54: α={alpha}, ε={epsilon}, Seed={seed}, {comp_str} Compression ")
                print(f"{'='*80}\n")
                
                set_seed(seed)
                
                data_loader = UCIHARDataLoader(DATA_PATH)
                client_loaders, test_loader, input_dim, num_classes = data_loader.get_federated_dataloaders(
                    num_clients=EXPERIMENT_CONFIG['num_clients'],
                    alpha=alpha,
                    batch_size=EXPERIMENT_CONFIG['batch_size'],
                    seed=seed
                )
                
                history, best_acc, test_loss, test_acc, test_f1, predictions, true_labels, total_comm = federated_training_dp(
                    client_loaders=client_loaders,
                    test_loader=test_loader,
                    input_dim=input_dim,
                    num_classes=num_classes,
                    num_rounds=EXPERIMENT_CONFIG['num_rounds'],
                    clients_per_round=EXPERIMENT_CONFIG['clients_per_round'],
                    learning_rate=EXPERIMENT_CONFIG['learning_rate'],
                    weight_decay=EXPERIMENT_CONFIG['weight_decay'],
                    epsilon=epsilon,
                    delta=EXPERIMENT_CONFIG['delta'],
                    clip_norm=EXPERIMENT_CONFIG['clip_norm'],
                    use_compression=use_compression,
                    compression_ratio=EXPERIMENT_CONFIG['compression_ratio'],
                    device=device,
                    seed=seed,
                    verbose=True
                )
                
                result = {
                    'alpha': alpha,
                    'epsilon': epsilon,
                    'seed': seed,
                    'compression': use_compression,
                    'test_accuracy': test_acc,
                    'test_f1': test_f1,
                    'test_loss': test_loss,
                    'best_accuracy': best_acc,
                    'history': history,
                    'predictions': predictions,
                    'true_labels': true_labels,
                    'total_communication_bytes': total_comm
                }
                all_results.append(result)
                
                comm_mb = total_comm / (1024 * 1024)
                print(f"\n{'-'*80}")
                print(f"Results: Acc={test_acc:.4f}, F1={test_f1:.4f}, Comm={comm_mb:.2f} MB")
                print(f"{'-'*80}")

print(f"\n{'='*80}")
print(" ALL 54 EXPERIMENTS COMPLETED SUCCESSFULLY ")
print(f"{'='*80}")

## 10. Results Analysis

In [ ]:
# Organize results
results_by_config = {}
for alpha in EXPERIMENT_CONFIG['alpha_values']:
    for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
        for comp in [False, True]:
            key = (alpha, epsilon, comp)
            results_by_config[key] = [r for r in all_results 
                                      if r['alpha'] == alpha and r['epsilon'] == epsilon and r['compression'] == comp]

print("\n" + "="*80)
print(" FL + DP + COMPRESSION ABLATION - SUMMARY ")
print("="*80)

# Summary by epsilon (privacy level)
for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
    print(f"\n{'='*80}")
    print(f" Privacy Budget: ε = {epsilon} (Lower ε = More Privacy) ")
    print(f"{'='*80}")
    
    for alpha in EXPERIMENT_CONFIG['alpha_values']:
        print(f"\n  Alpha = {alpha}:")
        
        for comp in [False, True]:
            results = results_by_config[(alpha, epsilon, comp)]
            accs = [r['test_accuracy'] for r in results]
            comms = [r['total_communication_bytes'] / (1024**2) for r in results]
            
            comp_str = "WITH Compression" if comp else "WITHOUT Compression"
            print(f"    {comp_str}:")
            print(f"      Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
            print(f"      Comm (MB): {np.mean(comms):.2f} ± {np.std(comms):.2f}")

# Create detailed results table
results_df = pd.DataFrame([{
    'Alpha': r['alpha'],
    'Epsilon': r['epsilon'],
    'Seed': r['seed'],
    'Compression': 'Yes' if r['compression'] else 'No',
    'Accuracy': f"{r['test_accuracy']:.4f}",
    'F1': f"{r['test_f1']:.4f}",
    'Comm (MB)': f"{r['total_communication_bytes']/(1024**2):.2f}"
} for r in all_results])

print("\n" + "="*80)
print("Sample Results (First 10):")
print(results_df.head(10).to_string(index=False))

## 11. Comprehensive Visualizations

### 11.1 Privacy-Utility Trade-off

In [ ]:
# Privacy-Utility trade-off: Epsilon vs Accuracy
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, alpha in enumerate(EXPERIMENT_CONFIG['alpha_values']):
    for comp in [False, True]:
        epsilons = []
        mean_accs = []
        std_accs = []
        
        for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
            results = results_by_config[(alpha, epsilon, comp)]
            accs = [r['test_accuracy'] for r in results]
            epsilons.append(epsilon)
            mean_accs.append(np.mean(accs))
            std_accs.append(np.std(accs))
        
        label = 'With Compression' if comp else 'No Compression'
        marker = '^' if comp else 'o'
        axes[i].errorbar(epsilons, mean_accs, yerr=std_accs, label=label, 
                        marker=marker, markersize=8, linewidth=2, capsize=5, alpha=0.8)
    
    axes[i].set_xlabel('Privacy Budget (ε)', fontsize=11)
    axes[i].set_ylabel('Test Accuracy', fontsize=11)
    axes[i].set_title(f'Alpha = {alpha}', fontsize=12, fontweight='bold')
    axes[i].legend(fontsize=10)
    axes[i].grid(True, alpha=0.3)
    axes[i].invert_xaxis()  # Lower epsilon (more privacy) on right

plt.suptitle('Privacy-Utility Trade-off (Lower ε = More Privacy)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('privacy_utility_tradeoff.png', dpi=300, bbox_inches='tight')
plt.show()

### 11.2 Communication Cost by Privacy Level

In [ ]:
# Communication cost comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, epsilon in enumerate(EXPERIMENT_CONFIG['epsilon_values']):
    alpha_labels = [f'α={alpha}' for alpha in EXPERIMENT_CONFIG['alpha_values']]
    
    no_comp_comms = [np.mean([r['total_communication_bytes']/(1024**2) 
                              for r in results_by_config[(alpha, epsilon, False)]]) 
                     for alpha in EXPERIMENT_CONFIG['alpha_values']]
    with_comp_comms = [np.mean([r['total_communication_bytes']/(1024**2) 
                                for r in results_by_config[(alpha, epsilon, True)]]) 
                       for alpha in EXPERIMENT_CONFIG['alpha_values']]
    
    x = np.arange(len(alpha_labels))
    width = 0.35
    
    axes[i].bar(x - width/2, no_comp_comms, width, label='No Compression', alpha=0.8, color='steelblue')
    axes[i].bar(x + width/2, with_comp_comms, width, label='With Compression', alpha=0.8, color='coral')
    
    axes[i].set_xlabel('Non-IID Level', fontsize=11)
    axes[i].set_ylabel('Communication (MB)', fontsize=11)
    axes[i].set_title(f'ε = {epsilon}', fontsize=12, fontweight='bold')
    axes[i].set_xticks(x)
    axes[i].set_xticklabels(alpha_labels)
    axes[i].legend(fontsize=10)
    axes[i].grid(True, alpha=0.3, axis='y')

plt.suptitle('Communication Cost by Privacy Level', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('communication_by_privacy.png', dpi=300, bbox_inches='tight')
plt.show()

### 11.3 Accuracy Heatmaps

In [ ]:
# Heatmaps: Epsilon vs Alpha for accuracy
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, comp in enumerate([False, True]):
    # Prepare heatmap data
    heatmap_data = np.zeros((len(EXPERIMENT_CONFIG['epsilon_values']), 
                             len(EXPERIMENT_CONFIG['alpha_values'])))
    
    for i, epsilon in enumerate(EXPERIMENT_CONFIG['epsilon_values']):
        for j, alpha in enumerate(EXPERIMENT_CONFIG['alpha_values']):
            results = results_by_config[(alpha, epsilon, comp)]
            heatmap_data[i, j] = np.mean([r['test_accuracy'] for r in results])
    
    sns.heatmap(heatmap_data, annot=True, fmt='.4f', cmap='RdYlGn', 
                xticklabels=[f'α={a}' for a in EXPERIMENT_CONFIG['alpha_values']],
                yticklabels=[f'ε={e}' for e in EXPERIMENT_CONFIG['epsilon_values']],
                ax=axes[idx], vmin=0.8, vmax=1.0, cbar_kws={'label': 'Accuracy'})
    
    title = 'With Compression' if comp else 'Without Compression'
    axes[idx].set_title(f'Accuracy: {title}', fontsize=13, fontweight='bold')
    axes[idx].set_xlabel('Non-IID Level', fontsize=11)
    axes[idx].set_ylabel('Privacy Budget', fontsize=11)

plt.suptitle('Accuracy Heatmap: Privacy vs Data Heterogeneity', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('accuracy_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

### 11.4 Training Curves Comparison

In [ ]:
# Training curves for different epsilon values (alpha=0.3, first seed)
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

alpha = 0.3  # Middle alpha value
seed = 42    # First seed

for i, epsilon in enumerate(EXPERIMENT_CONFIG['epsilon_values']):
    # Get results for this epsilon
    no_comp = [r for r in all_results if r['alpha']==alpha and r['epsilon']==epsilon 
               and r['seed']==seed and r['compression']==False][0]
    with_comp = [r for r in all_results if r['alpha']==alpha and r['epsilon']==epsilon 
                 and r['seed']==seed and r['compression']==True][0]
    
    rounds = np.arange(len(no_comp['history']['test_acc']))
    
    # Accuracy
    axes[0, i].plot(rounds, no_comp['history']['test_acc'], label='No Compression', linewidth=2)
    axes[0, i].plot(rounds, with_comp['history']['test_acc'], label='With Compression', linewidth=2)
    axes[0, i].set_xlabel('Round', fontsize=10)
    axes[0, i].set_ylabel('Test Accuracy', fontsize=10)
    axes[0, i].set_title(f'ε = {epsilon}', fontsize=11, fontweight='bold')
    axes[0, i].legend(fontsize=9)
    axes[0, i].grid(True, alpha=0.3)
    
    # Communication
    axes[1, i].plot(rounds, np.array(no_comp['history']['communication_bytes'])/(1024**2), 
                   label='No Compression', linewidth=2)
    axes[1, i].plot(rounds, np.array(with_comp['history']['communication_bytes'])/(1024**2), 
                   label='With Compression', linewidth=2)
    axes[1, i].set_xlabel('Round', fontsize=10)
    axes[1, i].set_ylabel('Cumulative Comm (MB)', fontsize=10)
    axes[1, i].legend(fontsize=9)
    axes[1, i].grid(True, alpha=0.3)

plt.suptitle(f'Training Curves (α={alpha}, Seed={seed})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('training_curves_dp.png', dpi=300, bbox_inches='tight')
plt.show()

### 11.5 Compression Savings Analysis

In [ ]:
# Calculate compression savings for each configuration
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, alpha in enumerate(EXPERIMENT_CONFIG['alpha_values']):
    epsilons = []
    savings = []
    acc_diffs = []
    
    for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
        no_comp_comm = np.mean([r['total_communication_bytes'] 
                                for r in results_by_config[(alpha, epsilon, False)]])
        with_comp_comm = np.mean([r['total_communication_bytes'] 
                                  for r in results_by_config[(alpha, epsilon, True)]])
        
        no_comp_acc = np.mean([r['test_accuracy'] 
                               for r in results_by_config[(alpha, epsilon, False)]])
        with_comp_acc = np.mean([r['test_accuracy'] 
                                 for r in results_by_config[(alpha, epsilon, True)]])
        
        epsilons.append(epsilon)
        savings.append((1 - with_comp_comm / no_comp_comm) * 100)
        acc_diffs.append((with_comp_acc - no_comp_acc) * 100)
    
    # Savings
    ax1 = axes[i]
    color = 'tab:blue'
    ax1.bar(epsilons, savings, alpha=0.7, color=color, label='Comm Savings')
    ax1.set_xlabel('Privacy Budget (ε)', fontsize=11)
    ax1.set_ylabel('Communication Savings (%)', fontsize=11, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_title(f'Alpha = {alpha}', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Accuracy difference
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.plot(epsilons, acc_diffs, marker='o', color=color, linewidth=2, 
            markersize=8, label='Acc Difference')
    ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax2.set_ylabel('Accuracy Difference (%)', fontsize=11, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

plt.suptitle('Compression Impact: Savings vs Accuracy', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('compression_impact.png', dpi=300, bbox_inches='tight')
plt.show()

### 11.6 Confusion Matrix (Best Configuration)

In [ ]:
# Find best configuration overall
best_result = max(all_results, key=lambda x: x['test_accuracy'])

print(f"\nBest Configuration:")
print(f"  Alpha: {best_result['alpha']}")
print(f"  Epsilon: {best_result['epsilon']}")
print(f"  Seed: {best_result['seed']}")
print(f"  Compression: {'Yes' if best_result['compression'] else 'No'}")
print(f"  Accuracy: {best_result['test_accuracy']:.4f}")
print(f"  F1 Score: {best_result['test_f1']:.4f}")

# Plot confusion matrix
cm = confusion_matrix(best_result['true_labels'], best_result['predictions'])

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=data_loader.activity_labels.values(),
            yticklabels=data_loader.activity_labels.values(),
            cbar_kws={'label': 'Count'})

plt.title(f'Confusion Matrix - Best Config\n'
         f'α={best_result["alpha"]}, ε={best_result["epsilon"]}, '
         f'Comp={"Yes" if best_result["compression"] else "No"}\n'
         f'Acc: {best_result["test_accuracy"]:.4f}',
         fontsize=13, fontweight='bold')
plt.ylabel('True Label', fontsize=11)
plt.xlabel('Predicted Label', fontsize=11)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('confusion_matrix_best.png', dpi=300, bbox_inches='tight')
plt.show()

## 12. Save Results

In [ ]:
os.makedirs('results/fedavg_dp_compression', exist_ok=True)

# Calculate summary statistics
summary_by_config = {}
for alpha in EXPERIMENT_CONFIG['alpha_values']:
    for epsilon in EXPERIMENT_CONFIG['epsilon_values']:
        for comp in [False, True]:
            key = f"alpha_{alpha}_epsilon_{epsilon}_comp_{comp}"
            results = results_by_config[(alpha, epsilon, comp)]
            
            summary_by_config[key] = {
                'mean_accuracy': float(np.mean([r['test_accuracy'] for r in results])),
                'std_accuracy': float(np.std([r['test_accuracy'] for r in results])),
                'mean_f1': float(np.mean([r['test_f1'] for r in results])),
                'mean_communication_mb': float(np.mean([r['total_communication_bytes']/(1024**2) for r in results]))
            }

output = {
    'experiment_config': EXPERIMENT_CONFIG,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'device': str(device),
    'summary_by_config': summary_by_config,
    'best_overall': {
        'alpha': best_result['alpha'],
        'epsilon': best_result['epsilon'],
        'seed': best_result['seed'],
        'compression': best_result['compression'],
        'test_accuracy': float(best_result['test_accuracy']),
        'test_f1': float(best_result['test_f1'])
    },
    'runs': [
        {
            'alpha': r['alpha'],
            'epsilon': r['epsilon'],
            'seed': r['seed'],
            'compression': r['compression'],
            'test_accuracy': float(r['test_accuracy']),
            'test_f1': float(r['test_f1']),
            'communication_mb': float(r['total_communication_bytes']/(1024**2))
        }
        for r in all_results
    ]
}

with open('results/fedavg_dp_compression/results.json', 'w') as f:
    json.dump(output, f, indent=2)

results_df.to_csv('results/fedavg_dp_compression/summary.csv', index=False)

print("\nResults saved:")
print("="*60)
print("  - results/fedavg_dp_compression/results.json")
print("  - results/fedavg_dp_compression/summary.csv")
print("\nVisualizations saved:")
print("  - privacy_utility_tradeoff.png")
print("  - communication_by_privacy.png")
print("  - accuracy_heatmap.png")
print("  - training_curves_dp.png")
print("  - compression_impact.png")
print("  - confusion_matrix_best.png")
print("="*60)